In [4]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [5]:
load_dotenv(override=True)

True

# Step 1: Agent Workflow

In [6]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini"
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini"
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini"
)

In [ ]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well.

Navigating the complexities of SOC 2 compliance can be a daunting challenge for many organizations. At ComplAI, we understand that maintaining compliance while maximizing operational efficiency is a top priority for businesses like yours.

Our AI-powered SaaS tool simplifies the SOC 2 compliance process by automating documentation, tracking compliance metrics, and preparing you for audits with ease. With ComplAI, you can streamline your compliance efforts, reduce audit preparation time, and ensure you meet industry standards without the usual headaches.

I would love to schedule a brief call to discuss how ComplAI can support your compliance journey and help you stay ahead of the curve. Are you available for a quick chat this week?

Thank you for considering ComplAI. I look forward to the possibility of working together.

Best regards,

[Your Name]  
[Your Position]  


In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Simplify Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we help companies like yours streamline the SOC 2 compliance process.

Navigating compliance can often be a daunting task, consuming valuable time and resources. ComplAI leverages cutting-edge AI technology to simplify this journey, helping you prepare for audits with ease and confidence. Our platform not only automates documentation but also provides continuous monitoring to ensure you stay compliant year-round.

Here are a few key benefits our clients experience:

- **Effortless Tracking:** Keep all your compliance documentation organized and accessible.
- **Reduced Audit Time:** Our AI-driven tools significantly speed up the audit preparation process.
- **Ongoing Compliance Monitoring:** Stay ahead of changing regulations with real-time updates.

I would love to schedule a brief call to discuss how ComplAI 

In [ ]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Is Your SOC2 Compliance as Smooth as a Baby’s Bottom? 🍼

Hi [Recipient's Name],

I hope this email finds you well and that your coffee is as strong as your commitment to compliance! ☕️

Let’s face it—dealing with SOC2 compliance can feel like trying to understand a cat’s meow. It’s confusing, time-consuming, and let’s not even start on those audits! Enter ComplAI, your new best friend (don’t worry, our feelings won’t be hurt if you have other friends too).

With our AI-powered SaaS tool, we take the “ugh” out of compliance. Imagine having:

- A trusty sidekick that understands your unique needs
- Automated workflows that practically do the tango while ensuring compliance
- A dashboard that’s easier to navigate than a toddler in a candy store

Our platform isn’t just about surviving audits; it’s about thriving through them. Let us handle the paperwork while you focus on what you do best—running your business (and perhaps perfecting that coffee brewing techniqu

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: Use of tools

Now we will add a tool to the mix.

In [8]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [11]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [ ]:

"""
Send out an email with the given body to all sales prospects.

Args:
    body (str): The content of the email to be sent.

Returns:
    dict: A dictionary with the status of the email sending operation. 
          Example: {"status": "success"}

Example:
    response = send_email("Hello, this is a test email.")
    print(response)  # Output: {'status': 'success'}
"""
@function_tool
def send_email(body: str):
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("daddysbackpack@gmail.com")
    to_email = To("daddysbackpack@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}